In [1]:
import pandas as pd
import  sys 
sys.path.append("../src/")
import data_preparation as dp

In [2]:
df = pd.read_csv("../data/train.csv")

/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_32762/3597601614.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/train.csv")


In [3]:
df = df.drop(columns=['MMS', 'Vf', 'De', 'Ernedc (g/km)', 'Enedc (g/km)', 'z (Wh/km)', 'Erwltp (g/km)', 'r' , 'Status'])

In [4]:
df['Electric range (km)'].fillna(0, inplace=True)

In [5]:
exp = dp.DataPreparation(df)

In [6]:
exp.get_type_list()

In [7]:
exp.impute_numerical()

100%|██████████| 8/8 [00:42<00:00,  5.34s/it]
